In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import json

In [ ]:
# main page with letters

next_link = "https://marvelcinematicuniverse.fandom.com/wiki/Category:Characters"

characters_link_list = []
while next_link != None:
    response = requests.get(next_link)
    html_page = response.text
    html_tree = BeautifulSoup(html_page)
    results = html_tree.find_all("a", class_="category-page__member-link")
    next_link = html_tree.find(
        "a", class_="category-page__pagination-next wds-button wds-is-secondary")
    next_link = next_link["href"] if next_link != None else None

    for result in results:
        characters_link_list.append(
            [result.text, "https://marvelcinematicuniverse.fandom.com"+result["href"]])


In [26]:
lista_personaggi = pd.DataFrame(characters_link_list, columns=["Nome", "Link"])

In [12]:
lista_personaggi = pd.read_csv("./Datasets/list_of_characters_with_biography.csv")
lista_personaggi.head()

,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [16]:
lista_personaggi.head()


,Nome,Link,Biografia
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere..."
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L...."
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I..."
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome..."
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad..."


In [8]:
lista_personaggi.shape

(3747, 2)

In [3]:
def cerca_bio(page_tree):
    biography_list = []
    biography_initial = page_tree.find_all(id="Biography")
    description_ended = False
    if(biography_initial):
        next_span = biography_initial[0].parent.find_next()

        while not description_ended:
            if next_span.name != "a" and next_span.name != "sup":
                biography_list.append(next_span.text)

            next_span = next_span.find_next()
            if next_span.name == "h2":
                description_ended = True
            biography_list = list(dict.fromkeys(biography_list))
            lista_personaggi.at[idx, "Biografia"] = biography_list
    return biography_list

In [23]:
def cerca_serie(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "tv series":
            tag_lista_film = element

    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [22]:
def cerca_film(page_tree):
    movies_list = []
    tag_lista_film = None
    movies_initial = page_tree.find_all("div", class_="pi-item pi-data pi-item-spacing pi-border-color")
    for element in movies_initial:
        if element["data-source"] == "movie":
            tag_lista_film = element
    
    if tag_lista_film != None:
        links = tag_lista_film.find_all("i")
        for link in links:
            movies_list.append([link.text, link.find_next("a")["href"]])
    return movies_list

In [13]:
def cerca_relazione_recursive(list_item):
  if not list_item:
    return []
  next_items = list_item.find_all("li")
  if not next_items:
    return [list_item.text.split(" - ",1)]
  else:
    recursive_list = []
    for item in next_items:
      recursive_list += cerca_relazione_recursive(item)
    return recursive_list



In [14]:
def cerca_relazione(page_tree):
  relationship_initial_tag = page_tree.find(id="Relationships")
  if relationship_initial_tag:
    rel_table = relationship_initial_tag.find_next("table")
    relation_list = []
    if rel_table:
      for ul_list in rel_table.find_all("ul"):
        for list_item in ul_list.find_all("li"):
          relation_list += cerca_relazione_recursive(list_item)
    return relation_list


In [24]:
from IPython.display import clear_output
lista_personaggi = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv")
#lista_personaggi["Biografia"] = None
lista_personaggi["Film"] = None
lista_personaggi["Serie"] = None
# lista_personaggi["Relazioni"] = None

num_iterazioni = 0

for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
    character_page_response = requests.get(row["Link"])
    character_page_tree = BeautifulSoup(character_page_response.text)
    #biografia = cerca_bio(character_page_tree)
    film = cerca_film(character_page_tree)
    serie = cerca_serie(character_page_tree)
    # relazioni = cerca_relazione(character_page_tree)
    #lista_personaggi.at[idx, "Biografia"] = biografia
    lista_personaggi.at[idx, "Film"] = json.dumps(film)
    lista_personaggi.at[idx, "Serie"] = json.dumps(serie)
    # lista_personaggi.at[idx, "Relazioni"] = json.dumps(relazioni)


100%|██████████| 3746/3746 [11:27<00:00,  5.45it/s]


In [25]:
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo.csv", index=False)

In [37]:
lista_personaggi = pd.read_csv("./Datasets/scraping_personaggi_completo.csv")
lista_personaggi.head(20)
filt = lista_personaggi.query("Nome == 'Grandmother'")
print(len(filt["Relazioni"]))


0


In [38]:
lista_personaggi["Processed_Name"] = None
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  lista_personaggi.at[idx, "Processed_Name"]= row["Nome"].replace("'", " ").replace('"', " ").replace(" \u2020","")
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_name.csv", index=False)


100%|██████████| 3746/3746 [00:00<00:00, 21648.21it/s]


In [39]:
test =[['test','test']]
print(json.dumps(test))
lista_personaggi = pd.read_csv(
    "./Datasets/scraping_personaggi_completo_filtered_name.csv")
lista_personaggi["Filtered_relazioni"] = None
errors =0
for idx, row in tqdm(lista_personaggi.iterrows(), total=lista_personaggi.shape[0]):
  try:
    relations = json.loads(row["Relazioni"])
    filtered_relations = []
    for relation in relations:
        querystring = "Processed_Name == '%s'" % (
          relation[0].replace("'", " ").replace('"', " ").replace(" \u2020",""))
        if len(lista_personaggi.query(querystring)) != 0:
          filtered_relations.append(relation)

    lista_personaggi.at[idx, "Filtered_relazioni"] = filtered_relations
  except Exception as e:
    errors+=1
print("Errors %s"%(errors))
  
lista_personaggi.to_csv("./Datasets/scraping_personaggi_completo_filtered_relations.csv", index=False)


[["test", "test"]]


100%|██████████| 3746/3746 [00:47<00:00, 79.47it/s] 


Errors 171


In [40]:
lista_personaggi.head(20)

,Nome,Link,Biografia,Relazioni,Processed_Name,Film,Serie,Filtered_relazioni
0,A'Lars,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Life on Titan', ""A'Lars fathere...",[],A Lars,"[[""Avengers: Infinity War"", ""/wiki/Avengers:_I...",[],[]
1,A. Kennedy,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'First Encounter With S.H.I.E.L....",[],A. Kennedy,[],"[[""Agents of S.H.I.E.L.D."", ""/wiki/Agents_of_S...",[]
2,A.I.M. President,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Obadiah Stane contacted the A.I...",[],A.I.M. President,[],[],[]
3,Aaron (Apple Inc.),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Apple Inc.', 'Assisting Custome...",[],Aaron (Apple Inc.),"[[""Captain America: The Winter Soldier"", ""/wik...",[],[]
4,Aaron (Baby),https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', ""Sophie walked with her son crad...",[],Aaron (Baby),[],"[[""Jessica Jones"", ""/wiki/Jessica_Jones_(TV_se...",[]
5,Aaron James,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Court Case', ""With Matt Murdock...","[[""Father""], [""Mother""], [""Matt Murdock"", ""Law...",Aaron James,[],"[[""The Defenders"", ""/wiki/The_Defenders""], [""T...",[[Berkowitz]]
6,Aaron Martinez,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Aaron Martinez was a criminal, ...",[],Aaron Martinez,[],"[[""Jessica Jones"", ""/wiki/Jessica_Jones_(TV_se...",[]
7,Abby,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Terrigenesis', '\n""When the Mis...","[[""Jemma Simmons""], [""Tye \u2020"", ""Trainer""],...",Abby,[],"[[""Agents of S.H.I.E.L.D."", ""/wiki/Agents_of_S...","[[Jemma Simmons], [Tye †, Trainer], [Basha, Ow..."
8,Abby Rose,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'To be added\n', 'To be added']",[],Abby Rose,[],[],[]
9,Abe Brown,https://marvelcinematicuniverse.fandom.com/wik...,"['Biography', 'Decathlon Team', '\n""Fort Sumte...",[],Abe Brown,"[[""Spider-Man: Homecoming"", ""/wiki/Spider-Man:...",[],[]
